# Integer basics
## Preliminaries

Let's load the mp++ runtime, include the ``integer.hpp`` header and add a couple of ``using`` directives to reduce typing:

In [ ]:
#pragma cling add_include_path("/srv/conda/envs/notebook/include")
#pragma cling add_library_path("/srv/conda/envs/notebook/lib")
#pragma cling load("mp++")

#include <mp++/integer.hpp>

using namespace mppp::literals;
// We will be working with integers with 1
// limb of static storage.
using int_t = mppp::integer<1>;

Let's also include a few useful bits from the standard library:

In [ ]:
#include <ios>
#include <iostream>
#include <stdexcept>
#include <string>
#include <string_view>

using namespace std::literals;

## Constructing multiprecision integers

There are many ways to construct multiprecision integers. Default-construction initialises to zero:

In [ ]:
{
    int_t n;
    std::cout << "A default-constructed integer is " << n << '\n';
}

We can construct from fundamental C++ types:

In [ ]:
int_t{42}

In [ ]:
int_t{-123.456} // Construction from floating-point types truncates

> **NOTE**: all constructors, apart from the default, copy and move constructors, are ``explicit``, thus syntax such as
>
> ```c++
> int_t n = 5;
> ```
>
> will **not** work. Use direct initialisation instead:
>
> ```c++
> int_t n{5};
> ```
>
> Or ``auto`` (Python-style):
>
> ```c++
> auto n = int_t{5};
> ```

We can construct from string-like types (including ``char[]``, ``std::string`` and ``std::string_view``):

In [ ]:
int_t{"-44939921"}

In [ ]:
int_t{"77"s}

In [ ]:
int_t{"-44"sv}

Construction from string representations in bases other than 10 is supported:

In [ ]:
int_t{"1001001", 2} // Base 2

In [ ]:
int_t{"FFFG", 17} // Base 17

mp++ also provides user-defined literals. Here we use the ``_z1`` literal, which constructs integers with 1 limb of static storage from decimal, binary, octal or hexadecimal literals:

In [ ]:
-101_z1 // Decimal literal

In [ ]:
0b10101010_z1 // Binary literal

In [ ]:
07117432_z1 // Octal literal

In [ ]:
0xDEADD00D_z1 // Hex literal

> **NOTE**: throughout these tutorials, we will almost always use the ``_z1`` literal to construct integer values.

Assignment to multiprecision integers works as expected:

In [ ]:
{
    int_t n;
    n = 123;
    std::cout << n << '\n';
}

In [ ]:
{
    int_t n;
    n = -456.789; // Assignment from floating-point types truncates
    std::cout << n << '\n';
}

In [ ]:
{
    int_t n;
    n = "987654"; // Assignment from string assumes base 10
    std::cout << n << '\n';
}

## Converting multiprecision integers

We can convert multiprecision integers to other types:

In [ ]:
double{123_z1} // Explicit type conversion syntax

In [ ]:
static_cast<long long>(-456_z1) // static_cast syntax

> **NOTE**: all of ``integer``'s conversion operators are ``explicit``, thus syntax such as
>
> ```c++
> int n = 5_z1;
> ```
>
> will **not** work. Use direct initialisation instead:
>
> ```c++
> int n{5_z1};
> ```
>
> Or ``auto`` (Python-style):
>
> ```c++
> auto n = int{5_z1};
> ```


Conversion to C++ integral values will fail in case of overflow:

In [ ]:
try {
    static_cast<unsigned>(-1_z1);
} catch (const std::overflow_error &oe) {
    std::cerr << oe.what() << '\n';
}

If exceptions are to be avoided, we can use the ``mppp::get()`` conversion function instead:

In [ ]:
{
    std::cout << std::boolalpha;
    
    unsigned out = 42;
    // Attempt to convert -1 to unsigned,
    // storing the result of the conversion
    // in 'out'. mppp::get() will return
    // a boolean exit status.
    const bool res = mppp::get(out, -1_z1);
    
    std::cout << "Did the conversion succeed? " << res << '\n';
    std::cout << "The value of 'out' is still " << out << '\n';
}

Conversion to floating-point types might be subject to rounding:

In [ ]:
std::cout << std::setprecision(20) << float{37211191293_z1} << '\n';

Integers are contextually convertible to ``bool``, thus you can directly use them in ``if`` statements or in conditional operators:

In [ ]:
if (1_z1) {
    std::cout << "1 is true\n";
} else {
    std::cout << "1 is false\n";
}

In [ ]:
std::cout << "123 is " << (123_z1 ? "nonzero" : "zero") << '\n';